Maskiere die Emailadressen in der Spalte _LogonName_ der Tabelle _People_ mit der geeigneten Maskierungsfunktion.

In [ ]:
ALTER TABLE Application.People ALTER COLUMN LogonName ADD MASKED WITH (FUNCTION = 'email()');

Prüfe nun, ob die Daten mit dem User _ethan_ wirklich maskiert daherkommen.

In [ ]:
GRANT SELECT ON Application.People TO ethan;
EXECUTE AS USER = 'ethan';
SELECT LogonName FROM Application.People;
REVERT;

Verschlüssle die Werte in der Spalte _CustomerID_ der Tabelle _CustomerTransactions_ und speichere diese in einer separaten Spalte. Verwende dabei den bereits erstellten Database Master Key.

In [ ]:
CREATE CERTIFICATE CustTrans_CustomerID WITH SUBJECT = 'Customer Transactions CustomerID ';  
GO  

CREATE SYMMETRIC KEY CTC_Key_01 WITH ALGORITHM = AES_256 ENCRYPTION BY CERTIFICATE CustTrans_CustomerID;  
GO  
 
OPEN SYMMETRIC KEY CTC_Key_01 DECRYPTION BY CERTIFICATE CustTrans_CustomerID; 

ALTER TABLE Sales.CustomerTransactions
ADD EncryptedCustID VARBINARY(128);
GO

UPDATE Sales.CustomerTransactions  
SET EncryptedCustID = EncryptByKey(Key_GUID('CTC_Key_01'), convert(varchar, CustomerID));  
GO  

CLOSE SYMMETRIC KEY SSN_Key_01;
GO

Prüfe anschliessend, ob die neue, verschlüsselte Spalte wieder korrekt entschlüsselt werden kann.

In [ ]:
OPEN SYMMETRIC KEY CTC_Key_01 DECRYPTION BY CERTIFICATE CustTrans_CustomerID;
GO

SELECT CustomerID, EncryptedCustID AS 'Encrypted CustomerID',  
CONVERT(varchar, DecryptByKey(EncryptedCustID)) AS 'Decrypted  CustomerID'  
FROM Sales.CustomerTransactions;  
GO

CLOSE SYMMETRIC KEY SSN_Key_01;
GO